In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,9653
2,Huelva,Confirmados PDIA 14 días,1831
3,Huelva,Tasa PDIA 14 días,"358,4973264440237"
4,Huelva,Confirmados PDIA 7 días,711
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,13
629,Municipio de Huelva sin especificar,Total Confirmados,124
630,Municipio de Huelva sin especificar,Curados,84


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  9653.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  2718.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 678 personas en los últimos 7 días 

Un positivo PCR cada 254 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,9653.0,711.0,1831.0,510743.0,139.208956,358.497326,39.0
Huelva-Costa,5319.0,469.0,1179.0,288115.0,162.782222,409.211600,15.0
Sierra de Huelva-Andévalo Central,1239.0,36.0,102.0,67571.0,53.277294,150.952332,13.0
Condado-Campiña,2972.0,193.0,529.0,155057.0,124.470356,341.164862,11.0
Huelva (capital),2718.0,212.0,566.0,143663.0,147.567571,393.977573,8.0
Moguer,297.0,42.0,95.0,22088.0,190.148497,430.097791,6.0
Lucena del Puerto,74.0,16.0,34.0,3371.0,474.636606,1008.602788,4.0
Valverde del Camino,266.0,8.0,15.0,12820.0,62.402496,117.004680,4.0
Isla Cristina,299.0,44.0,79.0,21264.0,206.922498,371.519940,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Corteconcepción,18.0,1.0,7.0,548.0,182.481752,1277.372263,0.0
Lucena del Puerto,74.0,16.0,34.0,3371.0,474.636606,1008.602788,4.0
Niebla,112.0,18.0,32.0,4117.0,437.211562,777.264999,0.0
Lepe,846.0,75.0,204.0,27431.0,273.413292,743.684153,1.0
Palma del Condado (La),433.0,32.0,79.0,10761.0,297.370133,734.132516,-1.0
Ayamonte,309.0,50.0,115.0,20946.0,238.709061,549.030841,0.0
Rociana del Condado,108.0,12.0,41.0,7855.0,152.768937,521.960535,2.0
Paymogo,12.0,2.0,6.0,1159.0,172.562554,517.687662,0.0
San Bartolomé de la Torre,31.0,6.0,19.0,3736.0,160.599572,508.565310,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Almonte,427.0,4.0,61.0,24191.0,16.535075,252.159894,0.0,0.065574
Minas de Riotinto,63.0,1.0,12.0,3848.0,25.987526,311.850312,1.0,0.083333
Corteconcepción,18.0,1.0,7.0,548.0,182.481752,1277.372263,0.0,0.142857
Alosno,41.0,1.0,6.0,3923.0,25.490696,152.944175,0.0,0.166667
Bonares,40.0,2.0,12.0,6058.0,33.014196,198.085177,0.0,0.166667
Bollullos Par del Condado,229.0,5.0,24.0,14272.0,35.033632,168.161435,0.0,0.208333
Trigueros,126.0,3.0,13.0,7713.0,38.895371,168.546610,-1.0,0.230769
Beas,53.0,4.0,17.0,4257.0,93.962885,399.342260,2.0,0.235294
Gibraleón,338.0,5.0,19.0,12607.0,39.660506,150.709923,-2.0,0.263158
